This notebook is specifically designed to read in ENSDF files and output to xml file using the lvlspy API. Below we install and import the necessary python packages

In [25]:
#quietly install any missing packages that are NOT built in to python
import sys, subprocess, pkg_resources, io, requests
required = {'numpy','scipy', 'lvlspy'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    subprocess.check_call([sys.executable,'-m','pip','install','--quiet',*missing])


#Import the necessary libraries 
import re #library that handles regular expressions
import math

import numpy as np
import lvlspy.level as lv
import lvlspy.spcoll as lc
import scipy.special as spc
import lvlspy.species as ls 
import lvlspy.transition as lt

from zipfile import ZipFile
from gslconsts.consts import *

The below section contains all the functions required to read and set the relevant data from the appropriate ENSDF file. There is no need to modify anything here, so the reader can simply skip to the next cell to run things. Other than the xml, a log file will be created that includes any levels or transitions that weren't included because they were of the wrong format                                   

In [26]:
#the expressions used to calculate the magnetic and electric rates are 
#derived in the book Nuclear Physics by Irving Kaplan, 2nd Edition

def rate_mag(E_i, E_f, j, A):
    '''
    Calculates the transition rate between two levels where the transition is a magnetic multipole

    Args:
    - E_i: energy of the initial state (in keV)
    - E_f: energy of the final state (in keV)
    - j  : the angular momentum of the state
    - A  : mass number


    Returns:
    - The magnetic contribution to the Weisskopf estimate between the two states
    '''
    dE = E_i - E_f 

    S = (2 * (j + 1)/(j*np.power(spc.factorial2(2*j+1),2))) * np.power(3/(j + 3),2)

    return(0.55 * S  * np.power(A,-2/3)    
        * np.power(dE/197000.0,2*j + 1)
        * np.power(1.4 * np.power(A, 1.0/3.0), 2*j)
        * GSL_CONST_NUM_ZETTA
    )

def rate_elec(E_i, E_f, j, A):
    '''
    Calculates the transition rate between two levels where the transition is an electric multipole

    Args:
    - E_i: energy of the initial state (in keV)
    - E_f: energy of the final state (in keV)
    - j  : the angular momentum of the state
    - A  : mass number


    Returns:
    - The electric contribution to the Weisskopf estimate between the two states
    '''
    
    dE = E_i - E_f

    S = (2 * (j + 1)/(j*np.power(spc.factorial2(2*j+1),2))) * np.power(3/(j + 3),2)

    return ( 2.4 * S 
        * np.power(dE / 197000.0, 2 * j + 1)
        * np.power(1.4 * np.power(A, 1.0 / 3.0), 2 * j)
        * GSL_CONST_NUM_ZETTA
    )

def extract_reduced_tran_prob(mod_B):
    reduced_prob = []
    mods = mod_B.split('$')
    if mod_B == 'N/A':
        reduced_prob.append(mod_B)
        return reduced_prob

    if len(mods) == 1:
        sp_mods = mods[0].split()
        
        if len(sp_mods[2]) > 4:
            reduced_prob.append(sp_mods[2])
        else:
            reduced_prob.append(sp_mods[2]+'='+sp_mods[4])
    else:
        for i in range(len(mods)):
            sp_mods = mods[i].split()
            if i == 0:
                if len(sp_mods[2]) > 4:
                    reduced_prob.append(sp_mods[2])
                else:
                    reduced_prob.append(sp_mods[2]+'='+sp_mods[4])
            else:
                if len(sp_mods[0]) > 4:
                    reduced_prob.append(sp_mods[0])
                else:
                    reduced_prob.append(sp_mods[0]+'='+sp_mods[2])

    return reduced_prob

def weisskopf_estimate(E_i, E_f, J_i, J_f,P_i,P_f, A,mod_B,MR):
    """
    Calculates the Weisskopf estimate for a transition between two states.

    Args:
    - E_i: energy of the initial state (in keV)
    - E_f: energy of the final state (in keV)
    - J_i: total angular momentum of the initial state
    - J_f: total angular momentum of the final state
    - P_i: parity of initial state
    - P_f: parity of final state
    - A  : mass number
    - mod_B: a string indicating whether a reduced transition porbability
    - 'MR' : a string indicating the mixing ratio

    Returns:
    - The total estimate for the transition rate (in per second) using Weisskopf single partice estimate
    """
    
    # avoid type difference in range function, cast the sum and different to integers
    diff = int(J_i - J_f)
    sm = int(J_i + J_f)
    j = range(max(1,abs(diff)),sm + 1) #range of photon angular momentum

    W = 0
    red_mat = extract_reduced_tran_prob(mod_B)
    if MR != '':
        mr = float(MR)
 
    for i in range(len(red_mat)):
        mod_b = red_mat[i]
        for jj in j:
            if np.power(-1,jj)*P_i == P_f:
                if mod_b[1] == 'E' and int(mod_b[2]) == jj:
                    B = float(mod_b[5:len(mod_b)])
                    if mod_b[1:3] == 'E2' and MR !='':
                        B = B*np.power(mr,2)/(1.0 + np.power(mr,2))
                    r = rate_elec(E_i,E_f,jj,A)
                    W += r*B
                else:
                    r = rate_elec(E_i,E_f,jj,A)
                    W += r/10 #Weisskopf tends to over estimate by a factor of 10 on average

            else:
                if mod_b[1] == 'M' and int(mod_b[2]) == jj:
                    B = float(mod_b[5:len(mod_b)])
                    if mod_b[1:3] == 'M1' and MR != '':
                        B = B/(1.0 + np.power(mr,2))
                    r = rate_mag(E_i,E_f,jj,A)
                    W += r*B
                else:
                    r = rate_mag(E_i,E_f,jj,A)
                    W += r/10 #Weisskopf tends to over estimate by a factor of 10 on average
    
    return W

def get_mass_num_and_file_name(species):
    '''
    Takes the species input, extracts the mass number and appends 0 to the start of the number to determine which file to load

    Args:
    - species (str):specifies specified by the user

    Returns:
    - file_name (str) : ENSDF file to be loaded
    - A (int): the mass number
    - identifier array consiting of l_indentifier, g_identifier, and b_identifier where:
        - l_identifier (str): the key used to help signify which lines to be read while reading in levels data
        - g_identifier (str): the key used to help signify which lines to be read while reading in gamma data
        - b_identifier (str): the key used to determine the reduced matrix element in Weisskopf Units
    '''

    match = re.search(r"\d+",species)
    
    if match:
        A = int(match.group())
    else:
        raise ValueError("Invalid Atomic Species Symbol")

    if len(match.group()) == 1:

        file_name  = 'ensdf.00'+match.group()
        identifier = '  ' + species
            
    elif len(match.group()) == 2:

        file_name  = 'ensdf.0'+match.group()
        identifier = ' ' + species
        
    else:

        file_name  = 'ensdf.'+match.group()
        identifier = species
    
    sym_len = len(species.replace(match.group(),''))

    if sym_len == 1:
        
        l_identifier = identifier + '   L'
        g_identifier = identifier + '   G'
    

    else:
        
        l_identifier = identifier + '  L'
        g_identifier = identifier + '  G'

    b_identifier = identifier + 'B '
    lib_sp = species.replace(match.group(),'').lower() + str(A)
    return file_name, A, lib_sp,[l_identifier, g_identifier,b_identifier]



def extract_multi_parity(jpi):
    '''
    Takes jpi as the input and extracts the j and the parity and calculates the multiplicity

    Args:
    
    - jpi (str): specifies the j and parity of the level

    Returns:
    - multi (int) : the multiplicity of the level
    - parity (str): the parity of the level
    '''

    #first strip any available parentheses
    jpi = jpi.replace('(','')
    jpi = jpi.replace(')','')
    
    if '+' not in jpi and '-' not in jpi: 
        parity = '+'
        multi = int(2*eval(jpi) + 1)
    else:
        parity = jpi[-1]       
        multi = int(2*eval(jpi[0:-1]) + 1)

    return multi, parity

def get_level_and_tran_data(file_loc,identifiers,filler_flag,mass_num):
    '''
    Takes the file and species' name and extracts the level data from the file

    Args:
    - file_name: the ENSDF formatted file
    - speces: species specified by the user

    Returns
    - lvls: a 2D array containing the level data with the following structure 'Energy','Multiplicity','Parity' 
    - trans: a  2D array containing the indices of the level transitions and any experimental reduced matrix coefficient    
    '''
    #lvls format is (energy, multiplicity, parity,isomeric state, half life,multiplet or singlet)
    lvls  = []
    #trans format is (top level, bottom level, reduced matrix,multipolarity, mixing ratio)
    trans = []
    zero_counter = 0 #zero counter required as to only read in the adopted values
    A = ['X','Y','Z','U','V','W','A','B']
    with open(file_loc,'r') as f:

        for line in f:
            #reading in level
            if line.startswith(identifiers[0]):

               energy = line[9:19].strip()
               if energy[0] in A:
                   str_dummy = energy[0]+'+'
                   energy = energy.replace(str_dummy,'')
               if energy[-1] in A:
                   str_dummy = '+'+energy[-1]
                   energy = energy.replace(str_dummy,'') 
                               
               energy = float(energy) #strip the spaces and cast to float
               if energy == 0.0: zero_counter += 1
               if zero_counter == 2: break # do not continue reading forward
               
               jpi    = line[21:39].strip() #strip the trailing spaces
               iso    = line[77:79]         #isomer indicator
               t_half = line[39:49].strip() #level half life

               if filler_flag:
                 if jpi == '':
                    log_file.write(str(energy) + ' and its subsequent gamma records not added to list since no J^pi specified\n')
                    k = -2
    
                 elif 'TO' in jpi or ',' in jpi or ':' in jpi or 'OR' in jpi:
                    log_file.write(str(energy) + ' and its subsequent gamma records not added since multiplet state\n')
                    k = -2

                 else:
                    multiplet = 'singlet'
                    multi,parity = extract_multi_parity(jpi)
                    lvls.append([energy, multi, parity,iso,t_half,multiplet])
                    k = 2
               else:
                   k = 2
                   if jpi == '':
                       
                       if mass_num % 2 == 0:
                           jpi = '3+'
                           multiplet = 'singlet'
                           multi,parity = extract_multi_parity(jpi)
                           lvls.append([energy,multi,parity,iso,t_half,multiplet])
                           
                       else:
                           jpi = '7/2+'
                           multiplet = 'singlet'
                           multi,parity = extract_multi_parity(jpi)
                           lvls.append([energy,multi,parity,iso,t_half,multiplet])
                           

                   elif 'TO' in jpi or ',' in jpi or ':' in jpi or 'OR' in jpi:
                       jpi = jpi.replace('(','')
                       jpi = jpi.replace(')','')
                     

                       if 'TO' in jpi:
                           multiplet = jpi
                           jpi = jpi.split('TO')[0].strip()
                           multi,parity = extract_multi_parity(jpi)
                           lvls.append([energy,multi,parity,iso,t_half,multiplet])
                       elif ',' in jpi:
                           multiplet = jpi
                           jpi = jpi.split(',')[0].strip()
                           multi,parity = extract_multi_parity(jpi)
                           lvls.append([energy,multi,parity,iso,t_half,multiplet])
                       elif ':' in jpi:
                           multiplet = jpi
                           jpi = jpi.split(':')[0].strip()
                           multi,parity = extract_multi_parity(jpi)
                           lvls.append([energy,multi,parity,iso,t_half,multiplet])
                       else:
                           multiplet = jpi
                           jpi = jpi.split('OR')[0].strip()
                           multi,parity = extract_multi_parity(jpi)
                           lvls.append([energy,multi,parity,iso,t_half,multiplet])
                   else:
                        multiplet = 'singlet'
                        multi,parity = extract_multi_parity(jpi)
                        lvls.append([energy, multi, parity,iso,t_half,multiplet])                                            
                           
            #reading in gamma info
            if line.startswith(identifiers[1]) and k == 2:
                E_g = line[9:19].strip() #gamma ray energy
               
                if E_g[0] in A:
                    str_dummy = E_g[0]+'+'
                    E_g = E_g.replace(str_dummy,'')
                if E_g[-1] in A:
                    str_dummy = '+'+E_g[0] 
                    E_g = E_g.replace(str_dummy,'')

                if E_g.isalpha() or E_g == '':
                    log_file.write('Transition from '+str(lvls[-1][0]) +' not added since E_gamma not supplied')
                    continue
                
                E_g = float(E_g)
             
                dE = abs(lvls[-1][0] - E_g)
               
                index = -1
                B = 'N/A'
                for i in range(len(lvls)):
                    if math.isclose(abs(dE - lvls[i][0]),0.0,abs_tol= 1.0): index = i
        
                #reading in reduced matrix element in weiskopf units
                if index == -1:
                    log_file.write('connection not found from level ' + str(lvls[-1][0]) + ' downward with E_gamma = ' + str(E_g)+'\n')
                else:
                    M = line[31:41].strip() #multipolarity of transition
                    MR = line[41:49].strip() #Mixing ratio 'delta'
                    trans.append([len(lvls)-1,index,M,MR,B])
            
            if line.startswith(identifiers[2]) and k == 2: 
                trans[-1][4] = line        
            

    return lvls,trans

def fill_missing_transitions(sp,A):
    '''
    Takes in the species and the mass number and fills in missing transitions.

    Args:
    - sp: the species being handled
    - A: the mass number of the species

    Exits upon loop completion    
    '''
    levels = sp.get_levels()
    for i in range(1,len(levels)):
        for j in range(i):
            t_dummy = sp.get_level_to_level_transition(levels[i],levels[j])
            if t_dummy == None :
                E_i = levels[i].get_energy()
                E_j = levels[j].get_energy()

                J_i = int((levels[i].get_multiplicity() - 1)/2)
                J_j = int((levels[j].get_multiplicity() - 1)/2)

                P_i = levels[i].get_properties()['parity'] #parity of initial state
                P_j = levels[j].get_properties()['parity'] #parity of final state

                if P_i == '+':
                    P_i = 1
                else:
                    P_i = -1
        
                if P_j == '+':
                    P_j = 1
                else:
                    P_j = -1

                ein_A = weisskopf_estimate(E_i,E_j,J_i,J_j,P_i,P_j,A,'N/A','')
                t = lt.Transition(levels[i],levels[j],ein_A)
                t.update_properties({'Source' : 'Weisskopf Estimate' })
                sp.add_transition(t)

    return
    

def create_xml(species,directory,xml_name,ENSDF_only_flag):
    '''
    This function takes in the species array, directory path and xml name, loops through the species, extracts
    the level and transition data, and fills in the missing transitions

    Args:

    - species (array): an array of strings that contains the isotopes to be looped over
    - directory (str): the directory where the ENSDF files can be found
    - xmml_name (str): the name the user wishes to set the xml to
    '''
    global log_file
    s_coll = [] #initializes the species collection to be added to the xml
    log_file = open('logfile.txt','w')

    for i in range(len(species)):
        log_file.write('Log output for species: ' + species[i]+'\n')
        file_name, A, lib_sp,identifiers = get_mass_num_and_file_name(species[i])
        file_loc = directory + file_name
        levels,transitions = get_level_and_tran_data(file_loc,identifiers,ENSDF_only_flag,A)
        n_ensdf = len(transitions)

        #generating the levels
        levs = []
        for j in range(len(levels)):
            levs.append(lv.Level(levels[j][0],levels[j][1]))
            levs[j].update_properties({'parity' : levels[j][2]})
            levs[j].update_properties({'multiplet' : levels[j][-1]})
            if levels[j][4].isspace() == False:
                levs[j].update_properties({'half-life' : levels[j][4]})
            if levels[j][3].isspace() == False:
                levs[j].update_properties({'isomer' : levels[j][3].strip()})

    
        sp = ls.Species(lib_sp,levels = levs)

        #using the lvlspy api to help make Einstein A calculations and transition building easier
        lvs = sp.get_levels()

        for j in range(len(transitions)):
            E_i = lvs[transitions[j][0]].get_energy()
            E_f = lvs[transitions[j][1]].get_energy()

            J_i = (lvs[transitions[j][0]].get_multiplicity() - 1)/2
            J_f = (lvs[transitions[j][1]].get_multiplicity() - 1)/2

            p_i = lvs[transitions[j][0]].get_properties()['parity']
            p_f = lvs[transitions[j][1]].get_properties()['parity']

            if p_i == '+':
                p_i = 1
            else:
                p_i = -1
        
            if p_f == '+':
              p_f = 1
            else:
                p_f = -1

            ein_A = weisskopf_estimate(E_i,E_f,J_i,J_f,p_i,p_f,A,transitions[j][4],transitions[j][3])
        
            t = lt.Transition(lvs[transitions[j][0]],lvs[transitions[j][1]],ein_A)
            t.update_properties({'Source' : 'ENSDF File'})
            if transitions[j][2] != '':
                t.update_properties({'Transition Multipolarity': transitions[j][2]})
            if transitions[j][3] != '':
                t.update_properties({'Mixing Ratio': transitions[j][3]}) 
            
            sp.add_transition(t)
        #append the fully compiled list of levels and transitions to the species collection
        if not ENSDF_only_flag:
            fill_missing_transitions(sp,A)
        
        s_coll.append(sp)
        log_file.write('Log summary for: ' + species[i] + ': number of levels: ' + str(len(levels)) + 
                       ' ,number of ensdf transitions: ' + str(n_ensdf)+'\n')
        
         
    log_file.close()
    #add the array of species collection to a lvlspy collection
    species_collection = lc.SpColl(s_coll)

    #write collection to xml
    species_collection.write_to_xml(xml_name+'.xml')

    return

In [28]:
#The species array must follow the formatting found in ENSDF files, the nuclear mass followed by the
#Latin symbol in uppercase. You can specify as many isotopes as you want. The functions above will reformat
#the isotope to the appropriate libnucnet format

species = ['26AL','85KR']

#the ENSDF database revision downloaded and used is 230501 at the time of writing and was put in the same directory 
#as this notebook, but if the user preferes a different data set predownloaded and extracted somewhere else, all
#they would have to do is change the directory to that one and everything else will work no problem or update the link
#and directory name

zf = ZipFile(io.BytesIO(requests.get('https://www.nndc.bnl.gov/ensdfarchivals/distributions/dist23/ensdf_230501.zip').content))
directory = 'ensdf_files/'
zf.extractall(directory)

#finally, set the file name you wish to label the xml with, set the string without the extension
xml_name = 'nuc_data_test'

# This flag is a boolean. A quick and close look at the ENSDF files will show you that not all the levels have Gamma transitions
# to the levels below them. This flag will tell the notebook to either just read in the ENSDF data or fill in the blanks when 
# set to False
ENSDF_only_flag = False 

#this function will simply call the above function and create the xml
create_xml(species,directory,xml_name,ENSDF_only_flag)